In [ ]:
# Import Libraries
import matplotlib.pyplot as plt
import numpy as np
import torch
import pandas as pd

from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
from PIL import Image
from torch.autograd import Variable
from typing import Union
from PIL import Image
from torch import Tensor

In [ ]:
# SOURCE: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

# Set directory of testing folder
root_dir_test = ''

# Prelimnary data transformations done on testing dataset
data_transformation_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

# Apply preprocessing and load dataloader with described batch size
test_image_dataset = datasets.ImageFolder(root = root_dir_test, transform=data_transformation_test)
testloader = DataLoader(test_image_dataset, batch_size=16, shuffle=True)

In [ ]:
# Set model to be sent to cpu
device = torch.device("cpu")

# Load the resnet50 model
model = models.resnet50()

# Determine the classifier architecture for the model
num_inftr = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_inftr, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 10),
    nn.LogSoftmax(dim=1)
)

# Load the model in directory, set to cpu and evaluation mode
model.load_state_dict(torch.load('model.pth',map_location=torch.device('cpu')))
model.eval()

In [ ]:
# SOURCE: https://github.com/kvarun07/covid-19-detection/blob/main/Covid-19-detection.ipynb
# Get class names (MSIMUT, MSS)
class_names = testloader.dataset.classes

def show_images(images, labels, preds):
    """
    This function displays the images to provide a visualization of the data augmentations done on the training
    dataset.
    :param images: The current DataLoader of the image at which data augmentation has been done
    :param labels: The current label of the image
    :param preds: The predicted label of the image in training dataset
    :return: a subplot of 1 by 6 cancer images with their labels and predicted labels
    """
    plt.figure(figsize=(8,4))
    for i, image in enumerate(images):
        if i < 5:
            plt.subplot(1, 6, i+1, xticks=[], yticks=[])

            # Convert to from tensor to numpy by taking its transpose
            image = image.numpy().transpose((1, 2, 0))
            
            # Denormalise image to show, as images were normalised earlier
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            
            image = image*std + mean
            image = np.clip(image, 0.,1.)
            plt.imshow(image)
            
            colour = 'green' if preds[i] == labels[i] else 'red'
            plt.xlabel(f'{class_names[int(labels[i].numpy())]}')
            plt.ylabel(f'{class_names[int(preds[i].numpy())]}', color=colour)
    plt.tight_layout()        
    plt.show()

In [ ]:
# SOURCE: https://github.com/kvarun07/covid-19-detection/blob/main/Covid-19-detection.ipynb
def show_preds():
    """
    This function sets the model to evaluation mode to retrieve batches of testing data and predict its
    class labels using the model loaded earlier
    """
    model.eval()    # set to evaluation mode
    images, labels = next(iter(testloader))
    outputs = model(images)
    _ , preds = torch.max(outputs, 1)
    show_images(images, labels, preds)

In [ ]:
# Call show_preds() function to plot a 1 by 5 subplot of images alongside its labels and predicted result
show_preds()

In [ ]:
# SOURCE: https://www.kaggle.com/yangdliu/notebook285235a998
def to_numpy(tensor: Union[Tensor, Image.Image, np.array]) -> np.ndarray:
    """
    This function is used to convert the format of the image to the desired format
    """
    if type(tensor) == np.array or type(tensor) == np.ndarray:
        return np.array(tensor)
    elif type(tensor) == Image.Image:
        return np.array(tensor)
    elif type(tensor) == Tensor:
        return tensor.cpu().detach().numpy()
    else:
        raise ValueError()

# Import packages to plot confusion matrix and AUC Score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

def test_label_predictions(model, device, test_loader):
    """
    This function is used to calculate the confusion matrix and plot the AUC score of the given model
    :params model: the loaded trained resnet50 model
    :params device: the current location of the model, whether its sent to CPU or GPU
    :test_loader: the testloader of the transformed batch of testing dataset
    :return: the confusion matrix of model and AUC score 
    """
    model.eval()
    actuals = []
    predictions = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            prediction = outputs.argmax(dim=1, keepdim=True)
            actuals.extend(to_numpy(labels.view_as(prediction)))
            predictions.extend(to_numpy(prediction))
            
    return [i.item() for i in actuals], [i.item() for i in predictions]

actuals, predictions = test_label_predictions(model, device, testloader)
print('Confusion matrix for resnet50: ')
print(confusion_matrix(actuals, predictions))
print('AUC score for model resnet50: '+str(roc_auc_score(actuals,predictions)))